<a href="https://colab.research.google.com/github/donrockvic/Tensorflow/blob/master/NLP_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

import tensorflow.keras.preprocessing

In [0]:
tf.__version__

'1.15.0'

In [0]:
tf.enable_eager_execution()

In [0]:
!pip install -q tensorflow_datasets

In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSGXYOE/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSGXYOE/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSGXYOE/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [0]:
training_sentences = []
training_labels = []

testing_sentences =[]
testing_labels = []

#
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [0]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [0]:
model = tf.keras.Sequential([                      
                             tf.keras.layers.Embedding(vocab_size, embedding_dim,input_length=max_length),
                             tf.keras.layers.Flatten(), 
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
"""
model = tf.keras.Sequential([                      
                             tf.keras.layers.Embedding(vocab_size, embedding_dim,input_length=max_length),
                             tf.keras.layers.GlobalAveragePooling1D(), 
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])


"""

"\nmodel = tf.keras.Sequential([                      \n                             tf.keras.layers.Embedding(vocab_size, embedding_dim,input_length=max_length),\n                             tf.keras.layers.GlobalAveragePooling1D(), \n                             tf.keras.layers.Dense(6, activation='relu'),\n                             tf.keras.layers.Dense(1, activation='sigmoid')\n])\n\n\n"

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epoch = 8
model.fit(padded,
          training_labels_final,
          epochs=num_epoch,
          validation_data = (testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/8
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


25000/25000 [==============================] - 10s 388us/sample - loss: 0.4935 - acc: 0.7434 - val_loss: 0.3510 - val_acc: 0.8418
Epoch 2/8
25000/25000 [==============================] - 8s 305us/sample - loss: 0.2479 - acc: 0.9032 - val_loss: 0.3647 - val_acc: 0.8392
Epoch 3/8
25000/25000 [==============================] - 7s 292us/sample - loss: 0.1095 - acc: 0.9708 - val_loss: 0.4402 - val_acc: 0.8274
Epoch 4/8
25000/25000 [==============================] - 7s 292us/sample - loss: 0.0311 - acc: 0.9950 - val_loss: 0.5225 - val_acc: 0.8220
Epoch 5/8
25000/25000 [==============================] - 7s 299us/sample - loss: 0.0079 - acc: 0.9995 - val_loss: 0.5956 - val_acc: 0.8235
Epoch 6/8
25000/25000 [==============================] - 7s 292us/sample - loss: 0.0022 - acc: 1.0000 - val_loss: 0.6437 - val_acc: 0.8253
Epoch 7/8
25000/25000 [==============================] - 7s 298us/sample - loss: 9.9742e-04 - acc: 1.0000 - val_loss: 0.6921 - val_acc: 0.8256
Epoch 8/8
25000/25000 [=========

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [0]:
reverse_word_index

{1: '<OOV>',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'br',
 9: 'in',
 10: 'it',
 11: 'i',
 12: 'this',
 13: 'that',
 14: 'was',
 15: 'as',
 16: 'for',
 17: 'with',
 18: 'movie',
 19: 'but',
 20: 'film',
 21: "'s",
 22: 'on',
 23: 'you',
 24: 'not',
 25: 'are',
 26: 'his',
 27: 'he',
 28: 'have',
 29: 'be',
 30: 'one',
 31: 'all',
 32: 'at',
 33: 'by',
 34: 'they',
 35: 'an',
 36: 'who',
 37: 'so',
 38: 'from',
 39: 'like',
 40: 'her',
 41: "'t",
 42: 'or',
 43: 'just',
 44: 'there',
 45: 'about',
 46: 'out',
 47: "'",
 48: 'has',
 49: 'if',
 50: 'some',
 51: 'what',
 52: 'good',
 53: 'more',
 54: 'very',
 55: 'when',
 56: 'she',
 57: 'up',
 58: 'can',
 59: 'b',
 60: 'time',
 61: 'no',
 62: 'even',
 63: 'my',
 64: 'would',
 65: 'which',
 66: 'story',
 67: 'only',
 68: 'really',
 69: 'see',
 70: 'their',
 71: 'had',
 72: 'were',
 73: 'me',
 74: 'well',
 75: 'we',
 76: 'than',
 77: 'much',
 78: 'been',
 79: 'get',
 80: 'bad',
 81: 'will',
 82: 'people',
 83: 'do',

In [0]:
import io

out_v = io.open('vecs.tsv','w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")


out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [0]:
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteTRNW26/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteTRNW26/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteTRNW26/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [0]:
imdb['train']

<DatasetV1Adapter shapes: ((?,), ()), types: (tf.int64, tf.int64)>

In [0]:
train_data, test_data = imdb['train'], imdb['test']

In [0]:
vocab_size = 10000
embedding_dim = 64
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'
tokenizer = info.features['text'].encoder

In [0]:
model = tf.keras.Sequential([                      
                             tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
                             tf.keras.layers.GlobalAveragePooling1D(), 
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 524,237
Trainable params: 524,237
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epoch = 10
model.fit(train_data,
          epochs=num_epoch,
          validation_data = test_data)

Train on None steps
Epoch 1/10
  25000/Unknown - 133s 5ms/step - loss: 0.6863 - acc: 0.5230

ValueError: ignored